In [ ]:
import codecs
import os
import zipfile
import time
import datetime
from gensim import models
#VocabProcessor is taken from the tensorflow.contrib learn.preprocessing.VocabularyProcessor and modified by us.
from VocabProcessor import VocabProcessor
import re
import itertools
from collections import Counter
from sklearn import metrics
from nltk.tokenize import sexpr
import numpy as np
from six.moves import urllib
import tensorflow as tf
sess = tf.InteractiveSession()
import tensorflow_fold as td
import pickle
import math

# Model Hyperparameters
# ==================================================
tf.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()

# pre-trained word embedding is trained with separate words.
# Original code was taken & modified from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
# ==================================================
def clean_str(string):
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r"  ", " ", string)
    return string.lower()

def separateFeatures(string):
    for line in string:
        line = clean_str(line)
        pid = line.split("\t")[0]
        sen = line.split("\t")[1]
        ddiCheck = line.split("\t")[2]
        ddiType = line.split("\t")[3]
        drug1 = line.split("\t")[4]
        drug1Name = line.split("\t")[5]
        drug1Type = line.split("\t")[6]
        drug2 = line.split("\t")[7]
        drug2Name = line.split("\t")[8]
        drug2Type = line.split("\t")[9]
        binaryParsedTree = line.split("\t")[10].strip()
        parsedWholeSen = line.split("\t")[11]
        
        yield binaryParsedTree, parsedWholeSen

# Loads DDI challenge'13 data from files
# The data should be preprocessed before.
# ==================================================
def load_data_and_labels(string):
    samples = list(open(string, "r").readlines())
    return list(separateFeatures(samples))

#detection
TrainFeatures = load_data_and_labels("data/DDItrain_verRP")
TestFeatures = load_data_and_labels("data/DDItest_verRP")

#this part is necessary for making a vocabulary
allSens1 = [Tf[1] for Tf in TrainFeatures] + [Tf[1] for Tf in TestFeatures]
splitted1 = [sentence.split(", ") for sentence in allSens1]
max_document_length1 = max(len(s) for s in splitted1)
vocab_proc1 = VocabProcessor(max_document_length1, tokenizer_fn="splitComma")
np.array(list(vocab_proc1.fit_transform([Tf[1] for Tf in TrainFeatures])))
np.array(list(vocab_proc1.fit_transform([Tf[1] for Tf in TestFeatures])))
vocab_proc1.vocabulary_.freeze()

print(len(vocab_proc1.vocabulary_._mapping))

In [ ]:
#load Word Embedding
bioEmbed = models.Word2Vec.load_word2vec_format('bioWordEmbedding/PubMed-and-PMC-w2v.bin', binary=True)
word_for_given_index1 = {}
for strWord in vocab_proc1.vocabulary_._mapping:
    word_for_given_index1[vocab_proc1.vocabulary_.get(strWord)] = strWord
count = 0

#initialize the whole word's vector with random numbers
#and if WE contains a word, replace the word's vector with the vector in WE.
init_range = math.sqrt(6.0 / (1 + 200))
embedding_for_given_index1 = np.random.uniform(-init_range,init_range,(len(vocab_proc1.vocabulary_), FLAGS.embedding_dim)).astype(np.float32)
for strIndex in range(len(word_for_given_index1)):
    strWord = word_for_given_index1.get(strIndex)
    if strWord in bioEmbed:
        embedding_for_given_index1[count] = bioEmbed[strWord]
    count = count + 1

#If we use the raw WE, the training process takes a lot of time
#                          due to the enormous size of vectors,
#we use the words appeared in the DDI corpus only to speed up the training.
#we save the selected WE words using pickle
ft = open('shorten_ensemble/pubpmc.pickle', 'wb')
pickle.dump(embedding_for_given_index1, ft)
ft.close()